Libraries

In [1]:
import os
import glob
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from transformers import ViTFeatureExtractor, ViTForImageClassification
from torchvision import models
import kagglehub
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Dataset Path

In [3]:
dataset_root = kagglehub.dataset_download("orvile/brain-cancer-mri-dataset")
data_dir = os.path.join(dataset_root, "Brain_Cancer raw MRI data")
data_dir = os.path.join(data_dir, "Brain_Cancer")

100%|██████████| 144M/144M [00:05<00:00, 25.5MB/s]

Extracting files...


Transformations and Dataset Loaders

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

dataset = ImageFolder(root=data_dir, transform=transform)
class_names = dataset.classes


Split into train/test

In [5]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [ ]:
# from collections import defaultdict
# import random
# random.seed(42)

# class_to_indices = defaultdict(list)
# for idx, (_, label) in enumerate(dataset.samples):
#     class_to_indices[label].append(idx)

# # Sample 100 train and 100 test indices per class
# train_indices = []
# test_indices = []

# for class_id, indices in class_to_indices.items():
#     random.shuffle(indices)
#     train_indices.extend(indices[:100])
#     test_indices.extend(indices[100:200])

# # Create subsets using the sampled indices
# from torch.utils.data import Subset

# train_dataset = Subset(dataset, train_indices)
# test_dataset = Subset(dataset, test_indices)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32)

CNN Model

In [7]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 54 * 54, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

Train CNN

In [8]:
cnn = SimpleCNN(num_classes=len(class_names)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=1e-4)

def train_model(model, loader):
    model.train()
    for epoch in range(3):
        running_loss = 0.0
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1} - Loss: {running_loss:.4f}")

train_model(cnn, train_loader)

def evaluate_model(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
    print("CNN Evaluation:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

evaluate_model(cnn, test_loader)

Epoch 1 - Loss: 99.1905
Epoch 2 - Loss: 78.2566
Epoch 3 - Loss: 70.3982
CNN Evaluation:
              precision    recall  f1-score   support

brain_glioma       0.70      0.86      0.77       604
 brain_menin       0.80      0.57      0.67       613
 brain_tumor       0.80      0.84      0.82       600

    accuracy                           0.76      1817
   macro avg       0.77      0.76      0.75      1817
weighted avg       0.77      0.76      0.75      1817



ViT Fine-Tuning

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
vit_model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=len(class_names)).to(device)

from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, subset):
        self.data = subset
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img, label = self.data[idx]
        return {"pixel_values": img, "label": label}

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

vit_train_dataset = CustomDataset(train_dataset)
vit_test_dataset = CustomDataset(test_dataset)

training_args = TrainingArguments(
    output_dir="./vit_output",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",
    remove_unused_columns=False
)

trainer = Trainer(
    model=vit_model,
    args=training_args,
    train_dataset=vit_train_dataset,
    eval_dataset=vit_test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
print("ViT Evaluation:")
trainer.evaluate()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:


Abort: 

SVM

In [6]:
def extract_features(dataloader):
    features = []
    labels = []
    with torch.no_grad():
        for imgs, lbls in dataloader:
            imgs = imgs.to(device)
            output = resnet(imgs).cpu().numpy()
            features.extend(output)
            labels.extend(lbls.numpy())
    return np.array(features), np.array(labels)

# Load ResNet once
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()
resnet = resnet.to(device)
resnet.eval()

# Extract features from train and test sets
svm_train_features, svm_train_labels = extract_features(train_loader)
svm_test_features, svm_test_labels = extract_features(test_loader)

# Train and evaluate SVM
svm = SVC(kernel='linear')
svm.fit(svm_train_features, svm_train_labels)
svm_preds = svm.predict(svm_test_features)

# Print metrics
print("SVM Evaluation:")
print(classification_report(svm_test_labels, svm_preds, target_names=class_names))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 152MB/s]


SVM Evaluation:
              precision    recall  f1-score   support

brain_glioma       0.96      0.95      0.95       604
 brain_menin       0.87      0.88      0.88       613
 brain_tumor       0.92      0.92      0.92       600

    accuracy                           0.92      1817
   macro avg       0.92      0.92      0.92      1817
weighted avg       0.92      0.92      0.92      1817

